# Tutorial of HOTRG algorithm with python

This notebook aims at providing the detailed explanations on how HOTRG works, with the step-by-step constructions. The code is organized into 3 classes:

* HOTRG
* XY
* Ising

where the main algorithm is written in HOTRG class. While XY and Ising classes are 2 examples which demonstrate how to compute the physical quantities through this method.  

## 1. Introduction
The main ideal of this alrorithm is based on the real space renormalization group (RG) procedure. 

In TRG language, the partition function is regarded as the tensor network. In partipular, for the 2-dimensional classical models, the tensor network is formed by the 4-legs tensor, $T_{l_i,r_i,u_i,d_i}$, which is living on each site of the 2D lattice. 

At each step of RG iterations, we would coarse grain 4 tensors as a cell into a new tensor. The procedure is taken by first contracting the X direction, then contract the Y direction. For example,

$$ M_{l,r,u,d}^{(n)} = \sum_i T_{l_1,r_1,u,i}^{(n)} T_{l_2,r_2,i,d}^{(n)}$$ where $l=l_1 \otimes l_2$ and $r=r_1 \otimes r_2$.

However, the bond dimensions, i.e. the number of indexes on the corresponding axis of tensor, would grow after we contract 2 tensors together. For instance, the $l$ and $r$ axes at the above equation have doubled thier bond dimensions. Therefore, we would perform the higher-order SVD (HOSVD) to truncate the bond dimensions.

$$T_{l',r',u,d}^{(n+1)}=\sum_{l,r} U_{l,l'}^{(n)}M_{l,r,u,d}^{(n)}U_{r,r'}^{(n)}$$ where $U^{(n)}$ is determined by the HOSVD.

Also, in practice, the amplitude of every tensor elements could go large (or small) after few RG iterations. Therefore, in order to keep it within the digit allowed in computers, we would normalize the tensor with its norm at each RG step. Also, these norms have to be stored in a histogram. We would add them back to the partition functions in the end of the calculations.

### 1.1 Ordering of the tensor $T$
In this note, the ordering of the tensor is counted counterclockwise. Namely,

<br>
<center> 
0 
<br>
|
<br>
1--T--3
<br>
|
<br>
2 
</center>

### 1.2 The oder after the tensor contraction
The way how python count the order after the tensor contraction is by first counting the axes of the first tensor with its original order, then count the second tensor with its orginal order. For example, if we are connecting bond 3 of tensor $A$ with bond 1 of tensor $B$

<br>
<center> 
0 
<br>
|
<br>
1--A--3
<br>
|
<br>
2 
</center>
and
<br>
<center> 
0 
<br>
|
<br>
1--B--3
<br>
|
<br>
2 
</center>

The resulting tensor is

<br>
<center> 
0 3
<br>
||
<br>
1--AB--5
<br>
||
<br>
2 4
</center>

Therefore, in order to keep the order being correctly in the way that we count the order counterclockwise. We would need to swap the axes. For example, for the above tensor, we need to swap:

$$1 \leftrightarrow 3$$
$$2 \leftrightarrow 3$$
$$0 \leftrightarrow 1$$

which leads to

<br>
<center> 
1 0
<br>
||
<br>
2--AB--5
<br>
||
<br>
3 4
</center>

then you can reshape it into a rank-4 tensor

<br>
<center> 
0
<br>
||
<br>
1--AB--3
<br>
||
<br>
2
</center>


### 1.3 References
Please also refer the following references:

[Ref.1 - Coarse-graining renormalization by higher-order singular value decomposition](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.86.045139)

[Ref.2 - Tensor renormalization group study of classical XY model on the square lattice](https://arxiv.org/abs/1309.4963)

[Ref.3 - Tensor network algorithm by coarse-graining tensor renormalization on finite periodic lattices](https://arxiv.org/abs/1510.03333)

## 2. Import packages
Packages which are used in this code.

In [95]:
import numpy as np
from itertools import product
from scipy.special import iv
import matplotlib.pyplot as plt

## 3. Class HOTRG
In this section, we would provide the detail explanations on the algorithm itself. There are 4 functions contained in this class:

* \__init\__(self,T0,D,maxstep)
* _HOSVD(self,M,chi,axes)
* T_norm(self,T)
* RG(self)



### 3.1 \__init\__(self,T0,D,maxstep)
Constructor of HOTRG class. 

* Arguments:

    * T0: ndarray <br>
        An initaial tensor for the RG iterations. The shape of this tensor is $(D,D,D,D)$.          
    * D: int     
        The bond dimensions.
    * maxstep: int  <br> 
        Maximum number of RG iterations.

In [96]:
class HOTRG:
    def __init__(self,T0,D,maxstep):
        self._init_tensor=T0
        self.D=D
        self.maxstep=maxstep

### 3.2 _HOSVD(self,M,chi,axes)
Performing the higher-order SVD to the tensor $M$,

$$ M_{xx'yy'}^{(n)} = \sum_{ijkl} S_{ijkl}U_{xi}^{L}U_{x'j}^{R}U_{yk}^{U}U_{y'l}^{D} $$

However, since we only need $U^L$ or $U^R$, only these two would be computed.

* Arguments:

    * M: ndarray <br>
        The tensor that you want to perform the HOSVD. The shape of this tensor is either $(D^2,D,D^2,D)$ or $(D,D^2,D,D^2)$.
    * chi: int <br>
        The bond dimensions to keep, which is D here.
    * axes: tuple(int,int) <br>
        The 2 axes that you want to perform the truncation via HOSVD.
    
* Returns:

    * U: ndarray <br>
        The left singular matrix with shape $(D^2,D)$.
        
<div class="alert alert-warning">
<b>NOTE</b> This function has already customized for the HOTRG algorithm, and is not suitable for the general situations.
</div>

In [97]:
    def _HOSVD(self,M,chi,axes):
        Us=[]; Ss=[]
        for n in axes:
            A=np.copy(M)
            if n>0:
                for roll in xrange(4-n):
                    A=np.rollaxis(A,-1)
            A=np.ndarray.reshape(A,(A.shape[0],int(A.size/A.shape[0])))
            x,s,y=np.linalg.svd(A,full_matrices=False) 
            Us.append(x); Ss.append(s)
            
        epsilon1=np.sum(np.square(np.delete(Ss[0],np.arange(chi))))
        epsilon2=np.sum(np.square(np.delete(Ss[1],np.arange(chi))))
        if epsilon1 < epsilon2:
            U=Us[0][:,0:chi]
        else:
            U=Us[1][:,0:chi]
        return U

This function can be seperated into 2 parts, with first compute the ordinary SVD on the 2 given axes, then perform the truncation by comparing the magnitude of $\epsilon_1$ and $\epsilon_2$, which are given by

$$ \epsilon_1 = \sum_{i>D} |S_{i,:,:,:}|^2 $$
and
$$ \epsilon_2 = \sum_{j>D} |S_{:,j,:,:}|^2 $$

They are basically the trucation error on the correponding axis, and we just choose to truncate the smaller one. 

### 3.3 T_norm(self,T)
Compute the norm of tensor $T$.

* Arguments:

    * T: ndarray <br>
        The tensor $T$ of which the shape is $(D,D,D,D)$.
        
* Returns:

    * norm: float <br>
        The norm of tensor $T$.

In [98]:
    def T_norm(self,T):
        norm=np.linalg.norm(np.ndarray.reshape(T,(self.D**2,self.D**2)))
        if np.isinf(norm) or norm<=0.0: raise ValueError('T_norm is {}'.format(norm))
        return norm

### 3.4 RG(self)
The main algorithm which performs the RG coarse graining procedure to tensor $T$.

* Returns:

    * T: ndarray <br>
        The tensor after maxstep of RG coarse graining.
    
    * norm_histo: list[float] <br>
        A list which stores all the normalization factors during the RG iterations.
    

In [99]:
    def RG(self):
        norm_histo=[] 
        T=self._init_tensor 
        for step in xrange(1,maxstep+1):
            print "RG step {}".format(step)
            # X direction
            M=np.tensordot(T,T,axes=(3,1))
            M=np.swapaxes(M,1,3)
            M=np.swapaxes(M,2,3)
            M=np.swapaxes(M,0,1)
            M=np.ndarray.reshape(M,(self.D**2,self.D,self.D**2,self.D))
            U=self._HOSVD(M,chi=self.D,axes=(0,2))                
            T=np.tensordot(np.tensordot(M,U,axes=(0,0)),U,axes=(1,0))
            T=np.swapaxes(T,0,2)
            T=np.swapaxes(T,1,2)
            T=np.swapaxes(T,2,3)
            # Y direction
            M=np.tensordot(T,T,axes=(2,0))
            M=np.swapaxes(M,3,4)
            M=np.swapaxes(M,2,4)
            M=np.swapaxes(M,4,5)
            M=np.ndarray.reshape(M,(self.D,self.D**2,self.D,self.D**2))
            U=self._HOSVD(M,chi=self.D,axes=(1,3))
            T=np.tensordot(np.tensordot(M,U,axes=(1,0)),U,axes=(2,0))
            T=np.swapaxes(T,1,2)                
            # normalized factor
            norm=self.T_norm(T)
            norm_histo.append(norm)
            T/=norm
        return T,norm_histo  

The coarse graining is performed first along the X axis. However, the bond dimensions on the Y direction is expanded from $D$ to $D^2$. Thus, we performed the HOSVD, and truncated these 2 bonds to dimensions $D$. The same procedure is taken for Y direction, then we would normalize the tensor $T$ with its norm due to the allowed numerical digits. These norms will be pushed back into the partion functions in the end of the calculations.

## 4. Class XY
There are 5 functions contained in this class:

* \__init\__(self,betas,h,D,maxstep)

* _init_tensor(self,beta)

* build_parti_func(self)

* free_energy(self,lnZs)

* energy(self,lnZs)

### 4.1 __init__(self,betas,h,D,maxstep)
Constructor of XY class.

* Arguments:

    * betas: array <br>
        An array which stores the range of inverse temperature.
    * h: float <br>
        The strength of external magnetic field of the XY model.
    * D: int <br>
        The bond dimensions.
    * maxstep: int <br>
        Maximum number of RG iterations.

In [100]:
class XY:
    def __init__(self,betas,h,D,maxstep):
        self.betas=betas
        self.h=h
        self.D=D
        self.maxstep=maxstep

### 4.2 _init_tensor(self,beta)
Creating the initial tensor $T$ of the XY model.

$$ T_{l,r,u,d} = \sqrt{I_l(\beta)I_r(\beta)I_u(\beta)I_d(\beta)} \, I_{l+r-u-d}(\beta h) $$

* Arguments:

    * beta: float <br>
        The inverse temperature.

* Returns：

    * T: ndarray <br>
        The tensor $T$ of the XY model.

In [101]:
    def _init_tensor(self,beta):
        T=np.zeros((self.D,self.D,self.D,self.D),dtype=float)
        for i,j,k,l in product(range(self.D),repeat=4):
            T[i,j,k,l]=np.sqrt(iv(i,beta)*iv(j,beta)*iv(k,beta)*iv(l,beta)) * iv(i+j-k-l,beta*self.h)
        T=np.swapaxes(T,1,2)
        return T

### 4.3 build_parti_func(self)
Compute the value of partion function from self.betas. In other words, this function returns the partition function as a function of inverse temperature.

* Returns:

    * lnZs: array <br>
        The logarithm of partition function as a function of inverse temperature.

In [111]:
    def build_parti_func(self):
        lnZs=[]
        for beta in self.betas:
            print "sim XY - beta= {}, h={}, D={}, maxstep={}".format(beta,self.h,self.D,self.maxstep)
            T0=self._init_tensor(beta)
            sim=HOTRG(T0,self.D,self.maxstep)
            T,norm_histo=sim.RG()
            lnZ=(np.sum(np.log(norm_histo)/np.logspace(1,self.maxstep,base=4,num=self.maxstep)) 
                    + np.log(sim.T_norm(T))/(4**self.maxstep))
            lnZs.append(lnZ)
        return np.array(lnZs)

Also, the norm of tensor T from each RG step is pushed back here by using the formula:

$$ \ln Z = \sum_{n=1}^{N} \frac{1}{4^n}\ln \lambda_n + \frac{\ln \lambda_{N}}{4^{N}} $$

where $\lambda_N$ is the norm of tensor $T$ after $N$ steps of RG iterations. 

### 4.4 free_energy(self,lnZs)
Compute the free energy by using the formula:

$$ F = - \frac{1}{\beta} \ln Z$$

* Arguments:

    * lnZs: array <br>
    
* Returns: 

    * F: array <br>

In [103]:
    def free_energy(self,lnZs):
        F=-lnZs/self.betas
        return F

### 4.5 energy(self,lnZs)
Compute the average (internal) energy by using the formula:

$$ E = - \frac{\partial}{\partial \beta} \ln Z $$

* Arguments:

    * lnZs: array

* Returns:

    * E: array

In [104]:
    def energy(self,lnZs):
        E=-np.diff(lnZs)/np.diff(self.betas)
        return E

## 5. Class Ising
There are 5 functions contained in this class:

* \__init\__(self,betas,maxstep)

* _init_tensor(self,beta)

* build_parti_func(self)

* free_energy(self,lnZs)

* energy(self,lnZs)

### 5.1 __init__(self,betas,maxstep)
Constructor of Ising class. Note that $D$ is 2 by default.

* Arguments:

    * betas: array <br>
        An array which stores the range of inverse temperature.
    * maxstep: int <br>
        Maximum number of RG iterations.

In [105]:
class Ising:
    def __init__(self,betas,maxstep):
        self.betas=betas
        self.D=2
        self.maxstep=maxstep

### 5.2 _init_tensor(self,beta)
Creating the initial tensor $T$ of the Ising model.

$$ T_{l,r,u,d} =  \sum_{\alpha} W_{\alpha,l}W_{\alpha,r}W_{\alpha,u}W_{\alpha,d} $$

where $W$ is a $2 \times 2$ matrix defined by

$$ W = \begin{pmatrix}
       \sqrt{\cosh(\beta)} & \sqrt{\sinh(\beta)} \\
       \sqrt{\cosh(\beta)} & -\sqrt{\sinh(\beta)} \\
       \end{pmatrix}$$

* Arguments:

    * beta: float <br>
        The inverse temperature.

* Returns：

    * T: ndarray <br>
        The tensor $T$ of the Ising model.

In [106]:
    def _init_tensor(self,beta):
        T=np.zeros((self.D,self.D,self.D,self.D),dtype=float)
        W=np.array([[np.sqrt(np.cosh(beta)),np.sqrt(np.sinh(beta))],
                     [np.sqrt(np.cosh(beta)),-np.sqrt(np.sinh(beta))]])
        for i,j,k,l in product(range(self.D),repeat=4):
            T[i,j,k,l]=np.sum(W[:,i]*W[:,j]*W[:,k]*W[:,l])
        return T   

### 5.3 build_parti_func(self)
Compute the value of partion function from self.betas. In other words, this function returns the partition function as a function of inverse temperature.

* Returns:

    * lnZs: array <br>
        The logarithm of partition function as a function of inverse temperature.

In [107]:
    def build_parti_func(self):
        lnZs=[]
        for beta in self.betas:
            print "sim XY - beta= {}, D={}, maxstep={}".format(beta,self.D,self.maxstep)
            T0=self._init_tensor(beta)
            sim=HOTRG(T0,self.D,self.maxstep)
            T,norm_histo=sim.RG()
            lnZ=(np.sum(np.log(norm_histo)/np.logspace(1,self.maxstep,base=4,num=self.maxstep)) 
                    + np.log(sim.T_norm(T))/(4**self.maxstep))
            lnZs.append(lnZ)
        return np.array(lnZs)

Also, the norm of tensor T from each RG step is pushed back here by using the formula:

$$ \ln Z = \sum_{n=1}^{N} \frac{1}{4^n}\ln \lambda_n + \frac{\ln \lambda_{N}}{4^{N}} $$

where $\lambda_N$ is the norm of tensor $T$ after $N$ steps of RG iterations. 

### 5.4 free_energy(self,lnZs)
Compute the free energy by using the formula:

$$ F = - \frac{1}{\beta} \ln Z$$

* Arguments:

    * lnZs: array <br>
    
* Returns: 

    * F: array <br>

In [108]:
    def free_energy(self,lnZs):
        F=-lnZs/self.betas
        return F

### 5.5 energy(self,lnZs)
Compute the average (internal) energy by using the formula:

$$ E = - \frac{\partial}{\partial \beta} \ln Z $$

* Arguments:

    * lnZs: array

* Returns:

    * E: array

In [109]:
    def energy(self,lnZs):
        E=-np.diff(lnZs)/np.diff(self.betas)
        return E

## 6. Main code
Plot the free energy $F$ and the average energy $E$ as a function of temperature $T$.

In [112]:
if __name__=='__main__':  
    
    Ts=np.linspace(0.1,2.0,30)
    betas=1./Ts
    h=0.0
    D=10
    maxstep=10
    
    #model=Ising(betas,maxstep)
    model=XY(betas,h,D,maxstep)
    lnZs=model.build_parti_func()
    F=model.free_energy(lnZs)
    E=model.energy(lnZs)
    
    plt.figure()
    plt.plot(Ts,F,linestyle="-",marker="o")
    plt.plot(Ts[:-1],E,linestyle="-",marker="o")
    plt.xlabel('T')
    plt.legend(('F/N','E/N'),loc=2,numpoints=1)
    plt.show()

AttributeError: XY instance has no attribute 'build_parti_func'